# Identifying mutation networks using SAMBAR
Author: 
Genís Calderer<sup>1</sup>

<sup>1</sup>Kuijjer Lab at Centre for Molecular Medicine Norway (NCMM).

## Introduction

SAMBAR is a method to identify subtypes of samples based on annotated mutation data. It uses gene mutation counts and membership of those genes in a set of pathways to summarize the mutation rate in a "pathway mutation score" thus despasifying the mutation data.
For each sample and each pathway a score is computed and it can be used to compare the samples to identify subtypes. The SAMBAR package uses binomial distance and hierarchical clustering to find subgroups of samples with similar pathway mutation patterns.
The method was developed and published in the following paper: Cancer subtype identification using somatic mutation data, Kuijjer ML, Paulson JN, Salzman P, Ding W, Quackenbush J, British Journal of Cancer (May 16, 2018), doi: 10.1038/s41416-018-0109-7, https://www.nature.com/articles/s41416-018-0109-7, BioRxiv, doi: https://doi.org/10.1101/228031

SAMBAR, or Subtyping Agglomerated Mutations By Annotation Relations, is a method to identify subtypes based on somatic mutation data. SAMBAR was used to identify mutational subtypes in 23 cancer types from The Cancer Genome Atlas (Kuijjer ML, Paulson JN, Salzman P, Ding W, Quackenbush J, *British Journal of Cancer* (May 16, 2018), doi: 10.1038/s41416-018-0109-7, https://www.nature.com/articles/s41416-018-0109-7, *BioRxiv*, doi: https://doi.org/10.1101/228031).

SAMBAR's input is a matrix that includes the number of non-synonymous mutations in a sample $i$
and gene $j$. SAMBAR first subsets these data to a set of 2,219 cancer-associated genes (optional) from the Catalogue Of Somatic Mutations In Cancer (COSMIC) and Östlund *et al*. (Network-based identification of novel cancer genes, 2010, *Mol Cell Prot*), or from a user-defined list. It then divides the number of non-synonymous mutations by the gene's length $L_j$, defined as the number of non-overlapping exonic base pairs of a gene. For each sample, SAMBAR then calculates the overall cancer-associated mutation rate by summing mutation scores in all cancer-associated genes $j'$. It removes samples for which the mutation rate is zero and divides the mutation scores the remaining samples by the sample's mutation rate, resulting in a matrix of mutation rate-adjusted scores $G$:

\begin{equation*}
G_{ij}=\frac{N_{ij}/L_{j}}{\displaystyle\sum_{j'}({N_{ij'}/L_{j'}})}.
\end{equation*}

The next step in SAMBAR is de-sparsification of these gene mutation scores (agglomerated mutations) into pathway mutation (annotation relation) scores. SAMBAR converts a (user-defined) gene signature (.gmt format) into a binary matrix $M$, with information of whether a gene $j$ belongs to a pathway $q$. It then calculates pathway mutation scores $P$ by correcting the sum of mutation scores of all genes in a pathway for the number of pathways $q'$ a gene belongs to, and for the number of cancer-associated genes present in that pathway:

\begin{equation*}
P_{iq}=\frac{\displaystyle\sum_{j \in q} G_{ij}/{\displaystyle\sum_{q'} M_{jq'}}}{\displaystyle\sum_{j} M_{jq}}
\end{equation*}

Finally, SAMBAR uses binomial distance to cluster the pathway mutation scores. The cluster dendrogram is then divided into $k$ groups (or a range of $k$ groups), and the cluster assignments are returned in a list.

This guide will use the toy data included in the SAMBAR package to showcase the usage of this package.

## 1. Importing SAMBAR from netZooPy

First, let's change the working directory.

In [ ]:
import os
os.chdir('../data')

In order to use the SAMBAR functions it has to be imported from the netZooPy as follows:

In [ ]:
from netZooPy import sambar

To see the parameters of the main function one can use the following line:

In [ ]:
help(sambar.sambar)

## 2. Selecting input files

The program requires a gene mutation dataset, a list of gene sizes, a list of cancer-associated genes (optional) and a list of pathways with its genes.

In [ ]:
# These are the names of the files of the toy dataset.
# The program by default runs with the toy data.
mut_file = "/opt/data/mut.ucec.csv"
cangenes = "/opt/data/genes.txt"
sign_file = "/opt/data/h.all.v6.1.symbols.gmt"
esize_file = "/opt/data/esizef.csv"

## 3. Run SAMBAR

The main SAMBAR function takes as input the filepaths of the datasets and returns a pathway score dataframe and a sample clustering dataframe for different cuts in the linkage tree.
It also outputs a csv file for the adjusted mutation scores, pathway scores and clustering.
The slow step in this method is the computation of the distance matrix, this matrix is also exported in case it's needed and rerunning the whole process is not wanted.
The function runs first the desparcification and then the clustering.

In [ ]:
pathway_scores, cluster_groups = sambar.sambar(mut_file,esize_file,cangenes,sign_file) 

In [ ]:
#pathway_scores, cluster_groups = sambar.sambar() 
#Runs with the default files from the netZooPy package, therefore
#it requires a user-specific installation of the netZooPy package (https://netzoo.github.io/contribute/install/).

## 4. Results

The pathway mutation scores and the sample groups are the output of the method.

In [ ]:
pathway_scores.head(10)

In [ ]:
cluster_groups.head()